# Objective: Create a tool to simulate end result of investing based on investments into certain stocks
### The tool should be able to take in an input bi-weekly or monthly amount and simulate what the final amount would have been if it was invested in one stock or multiple stocks

### This will be a streamlined notebook which will ahve everything in an input and output format with functions if necessary

In [1]:
#Loading required libraries

#Importing for data handling and math
import pandas as pd
import numpy as np
#Importing for downloading stock data and company financials if required
import yfinance as yf
from yahoofinancials import YahooFinancials
#Importing datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Input section

In [2]:
# This section will contain all the input from the user which has to be provided

#Input the list of Tickers to be downloaded
#Creating a dictionary of stocks and investment percentages
inv_mix_dict = {'Ticker':['AAPL'],
                'Inv_perc':[1]}

inv_mix_df = pd.DataFrame(inv_mix_dict)

print(inv_mix_df)

ticker_list = list(inv_mix_df.Ticker)

print(ticker_list)

#Taking in start and end dates of investments
start_dt = '2018-07-01'
end_dt = '2023-07-01'

#need to filter for days of investment i.e 15th and 30th
inv_days = [10,25]

#Regular Bi-weekly investment amount
reg_inv_amt = 4000

  Ticker  Inv_perc
0   AAPL         1
['AAPL']


## Data Gathering

In [10]:
#Creating a dataframe capturing stocks data as a timeseries
stock_df = pd.DataFrame()

for ticker in ticker_list:
    print('extracting {}'.format(ticker))
    ticker_df = yf.download(ticker,
                     start = start_dt,
                     end = end_dt)
    ticker_df = ticker_df.reset_index()
    ticker_df['Ticker'] = ticker
    if stock_df.empty:
        print("dataframe is empty")
        print("loading {} into dataframe".format(ticker))
        stock_df = ticker_df
    else:
        print("loading {} into dataframe".format(ticker))
        stock_df = pd.concat([stock_df,ticker_df])

print(stock_df.head(20))
print(stock_df.shape)

C:\Users\kruth\AppData\Local\Temp\ipykernel_29764\3624951928.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ticker_df = yf.download(ticker,
[*********************100%***********************]  1 of 1 completed

extracting AAPL
dataframe is empty
loading AAPL into dataframe
Price        Date      Close       High        Low       Open    Volume Ticker
Ticker                  AAPL       AAPL       AAPL       AAPL      AAPL       
0      2018-07-02  44.277512  44.305900  43.388083  43.482706  70925200   AAPL
1      2018-07-03  43.506348  44.459645  43.416457  44.421797  55819200   AAPL
2      2018-07-05  43.856449  44.095367  43.591514  43.823332  66416800   AAPL
3      2018-07-06  44.464367  44.573178  43.809121  43.861163  69940800   AAPL
4      2018-07-09  45.081772  45.105425  44.778988  44.826297  79026400   AAPL
5      2018-07-10  45.027374  45.247364  44.987158  45.112533  63756400   AAPL
6      2018-07-11  44.443092  44.892537  44.379223  44.589752  75326000   AAPL
7      2018-07-12  45.188232  45.278123  44.781365  44.833407  72164400   AAPL
8      2018-07-13  45.259186  45.379825  45.157467  45.200048  50055600   AAPL
9      2018-07-16  45.159847  45.571443  45.043936  45.304143  60172

## Feature Engineering
Adding other required features

In [11]:
#need to figure out how to calculate mean row by row
stock_df['Avg'] = stock_df[['High','Low']].mean(axis=1)

#Creating the day number marker as a column
stock_df['Day'] = stock_df.Date.dt.day

#removing multiindex from the dataframe
stock_df.columns = stock_df.columns.droplevel(1)

print(stock_df.head(5))

Price       Date      Close       High        Low       Open    Volume Ticker  \
0     2018-07-02  44.277512  44.305900  43.388083  43.482706  70925200   AAPL   
1     2018-07-03  43.506348  44.459645  43.416457  44.421797  55819200   AAPL   
2     2018-07-05  43.856449  44.095367  43.591514  43.823332  66416800   AAPL   
3     2018-07-06  44.464367  44.573178  43.809121  43.861163  69940800   AAPL   
4     2018-07-09  45.081772  45.105425  44.778988  44.826297  79026400   AAPL   

Price        Avg  Day  
0      43.846992    2  
1      43.938051    3  
2      43.843441    5  
3      44.191150    6  
4      44.942206    9  


In [12]:
#Now we need the price of the stocks during the latest day
#Filtering the stock data for the days of investment
stock_latest_dt_df = stock_df.groupby('Ticker').agg({'Date':'max'}).reset_index().rename(columns={'Date':'Latest_date'})
stock_latest_dt_df
#Bringing out the latest price for the stock
stock_latest_df = stock_latest_dt_df.merge(stock_df[['Ticker','Date','Avg']], left_on = ['Ticker','Latest_date'], right_on = ['Ticker','Date'], how = 'left').reset_index().rename(columns={'Avg':'Latest_avg'})
stock_latest_df

Price,index,Ticker,Latest_date,Date,Latest_avg
0,0,AAPL,2023-06-30,2023-06-30,190.958078


In [ ]:
#Creating a subset of the dataframe for days where the investments would have occured
stock_inv_days_df = stock_df[stock_df['Day'].isin(inv_days)]

#Adding a column to denote investment amount
stock_inv_days_df['Reg_inv_amt'] = reg_inv_amt

#Mapping this back to the the investment days dataframe
print(stock_inv_days_df.shape)
stock_inv_days_df = stock_inv_days_df.merge(inv_mix_df,on=['Ticker'],how='left')
print(stock_inv_days_df.shape)

#Amount invested
stock_inv_days_df['Stock_inv_amt'] = stock_inv_days_df['Reg_inv_amt']*stock_inv_days_df['Inv_perc']

#Quantity bought
stock_inv_days_df['Stock_qty'] = stock_inv_days_df['Stock_inv_amt']/stock_inv_days_df['Avg']

#Merging this back to the investment df
stock_inv_days_df = stock_inv_days_df.merge(stock_latest_df[['Ticker','Latest_avg']], on = ['Ticker'], how = 'left')
stock_inv_days_df['Latest_value'] = stock_inv_days_df['Stock_qty']*stock_inv_days_df['Latest_avg']

print(stock_inv_days_df.head(5))

## Return calculation

In [ ]:
#Final portfolio value is
portfolio_value = stock_inv_days_df['Latest_value'].sum().round(2)
print(portfolio_value)

In [ ]:
#Final amount invested is
invested_amt = stock_inv_days_df['Stock_inv_amt'].sum().round(2)
print(invested_amt)

In [ ]:
#Amount of money made
return_amt = (portfolio_value-invested_amt).round(2)
print(return_amt)

In [ ]:
#Percentage return
perc_return = ((portfolio_value/invested_amt-1)*100).round(2)
print(perc_return)

In [ ]:
#Amount of time passed
difference_in_years = relativedelta(datetime.strptime(end_dt,'%Y-%m-%d'), datetime.strptime(start_dt,'%Y-%m-%d')).years
difference_in_years